In [1]:
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path


In [2]:

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [3]:
import timm
# assert timm.__version__ == "0.3.2"  # version check
import timm.optim.optim_factory as optim_factory

c:\Users\joaoa\.conda\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:


import util.misc as misc
from util.misc import NativeScalerWithGradNormCount as NativeScaler
from util.custom_dataset import CustomDataset
import util.transform_npy as transform_npy


import models_mae

from engine_pretrain import train_one_epoch


In [9]:
def get_args_parser():
    # Create an argument parser named 'parser'
    parser = argparse.ArgumentParser('MAE Pre-training', add_help=False)
    # 'MAE Pre-training' is the description that will appear in the help message when the script is run
    # 'add_help=False' disables the default help option (-h or --help) to allow hierarchical argument structure

    # Define various arguments with their details using parser.add_argument()
    parser.add_argument('--batch_size', default=64, type=int,
                        help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus')
    
    parser.add_argument('--epochs', default=400, type=int)
    
    parser.add_argument('--accum_iter', default=1, type=int,
                        help='Accumulate gradient iterations (for increasing effective batch size under memory constraints)')

    # Model parameters
    parser.add_argument('--model', default='mae_vit_large_patch16', type=str, metavar='MODEL',
                        help='Name of model to train')

    parser.add_argument('--input_size', default=224, type=int,
                        help='Image input size')

    parser.add_argument('--mask_ratio', default=0.75, type=float,
                        help='Masking ratio (percentage of removed patches).')

    parser.add_argument('--norm_pix_loss', action='store_true',
                        help='Use normalized pixels (per patch) as targets for computing loss')
    parser.set_defaults(norm_pix_loss=False)  # Set default value for 'norm_pix_loss' as False

    # Optimizer parameters
    parser.add_argument('--weight_decay', type=float, default=0.05,
                        help='Weight decay (default: 0.05)')

    parser.add_argument('--lr', type=float, default=None, metavar='LR',
                        help='Learning rate (absolute lr)')
    
    parser.add_argument('--blr', type=float, default=1e-3, metavar='LR',
                        help='Base learning rate: absolute_lr = base_lr * total_batch_size / 256')
    
    parser.add_argument('--min_lr', type=float, default=0., metavar='LR',
                        help='Lower LR bound for cyclic schedulers that hit 0')

    parser.add_argument('--warmup_epochs', type=int, default=40, metavar='N',
                        help='Epochs to warm up LR')

    # Dataset parameters
    parser.add_argument('--data_path', default='/datasets01/imagenet_full_size/061417/', type=str,
                        help='Dataset path')

    parser.add_argument('--output_dir', default='./output_dir',
                        help='Path to save results, empty for no saving')
    
    parser.add_argument('--log_dir', default='./output_dir',
                        help='Path to TensorBoard log')
    
    parser.add_argument('--device', default='cuda',
                        help='Device to use for training / testing')
    
    parser.add_argument('--seed', default=0, type=int)
    
    parser.add_argument('--resume', default='',
                        help='Resume from checkpoint')

    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='Start epoch')
    
    parser.add_argument('--num_workers', default=10, type=int)
    
    parser.add_argument('--pin_mem', action='store_true',
                        help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')
    parser.add_argument('--no_pin_mem', action='store_false', dest='pin_mem')
    parser.set_defaults(pin_mem=True)  # Set default value for 'pin_mem' as True

    # Distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='Number of distributed processes')
    
    parser.add_argument('--local_rank', default=-1, type=int)
    
    parser.add_argument('--dist_on_itp', action='store_true')
    
    parser.add_argument('--dist_url', default='env://',
                        help='URL used to set up distributed training')

    return parser  # Return the fully configured argument parser


In [11]:
def main(args):
    # Initialize distributed mode if needed
    misc.init_distributed_mode(args)  # Initialize distributed training if required

    # Print job directory and arguments
    print('job dir: {}'.format(os.path.dirname(os.path.realpath(__file__))))
    print("{}".format(args).replace(', ', ',\n'))

    # Set the device for training (e.g., 'cuda' or 'cpu')
    device = torch.device(args.device)

    # Fix the random seed for reproducibility
    seed = args.seed + misc.get_rank()  # Combine provided seed and distributed rank
    torch.manual_seed(seed)  # Set PyTorch random seed
    np.random.seed(seed)  # Set NumPy random seed

    cudnn.benchmark = True  # Enable CuDNN benchmark mode for optimized performance


    transform_train = transforms.Compose([
        transform_npy.ResizeNpyWithPadding((args.input_size, args.input_size)),
        transform_npy.RandomHorizontalFlipNpy(),
        transform_npy.RandomRotationNpy(degrees=(-15, 15)),
        transforms.Lambda(lambda data: data.copy()),  # Copy the data to avoid grad error
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
])
    
    # Create a training dataset using the defined transformations
    dataset_train = CustomDataset(data_path=os.path.join(args.data_path, 'train'), transform=transform_train)
    print(dataset_train)

    # Configure data sampler for distributed training (if applicable)
    if True:  # args.distributed:
        num_tasks = misc.get_world_size()
        global_rank = misc.get_rank()
        sampler_train = torch.utils.data.DistributedSampler(
            dataset_train, num_replicas=num_tasks, rank=global_rank, shuffle=True
        )
        print("Sampler_train = %s" % str(sampler_train))
    else:
        sampler_train = torch.utils.data.RandomSampler(dataset_train)

    # Set up logging writer for TensorBoard
    if global_rank == 0 and args.log_dir is not None:
        os.makedirs(args.log_dir, exist_ok=True)
        log_writer = SummaryWriter(log_dir=args.log_dir)
    else:
        log_writer = None

    # Create data loader for training dataset
    data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )
    
    # Define the neural model using the specified architecture
    model = models_mae.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)

    # Move the model to the specified device
    model.to(device)

    # Calculate effective batch size for training
    eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()
    
    # Calculate learning rate based on batch size and base learning rate
    if args.lr is None:  # If only base_lr is specified
        args.lr = args.blr * eff_batch_size / 256

    # Print learning rate and other training settings
    print("base lr: %.2e" % (args.lr * 256 / eff_batch_size))
    print("actual lr: %.2e" % args.lr)
    print("accumulate grad iterations: %d" % args.accum_iter)
    print("effective batch size: %d" % eff_batch_size)

    # Set up DistributedDataParallel if using distributed training
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu], find_unused_parameters=True)
        model_without_ddp = model.module
    
    # Set weight decay for bias and norm layers following timm's recommendation
    param_groups = optim_factory.add_weight_decay(model_without_ddp, args.weight_decay)
    # Create an AdamW optimizer with specified parameters
    optimizer = torch.optim.AdamW(param_groups, lr=args.lr, betas=(0.9, 0.95))
    print(optimizer)
    # Create a loss scaler for mixed-precision training
    loss_scaler = NativeScaler()

    # Load model checkpoint and optimizer state if available
    misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

    # Start training loop for specified number of epochs
    print(f"Start training for {args.epochs} epochs")
    start_time = time.time()
    for epoch in range(args.start_epoch, args.epochs):
        # Set epoch for distributed training
        if args.distributed:
            data_loader_train.sampler.set_epoch(epoch)
        
        # Perform one epoch of training and get training statistics
        train_stats = train_one_epoch(
            model, data_loader_train,
            optimizer, device, epoch, loss_scaler,
            log_writer=log_writer,
            args=args
        )
        
        # Save model checkpoint and statistics periodically
        if args.output_dir and (epoch % 20 == 0 or epoch + 1 == args.epochs):
            misc.save_model(
                args=args, model=model, model_without_ddp=model_without_ddp, optimizer=optimizer,
                loss_scaler=loss_scaler, epoch=epoch)

        # Prepare log statistics for logging
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                        'epoch': epoch,}

        # Write log statistics to file if applicable
        if args.output_dir and misc.is_main_process():
            if log_writer is not None:
                log_writer.flush()
            with open(os.path.join(args.output_dir, "log.txt"), mode="a", encoding="utf-8") as f:
                f.write(json.dumps(log_stats) + "\n")

    # Calculate total training time and print
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))
